In [1]:
import numpy as np
import pandas as pd
import random
from pprint import pprint
import math

In [2]:
def metrics(ts_lb,answer):
    TN = 0
    TP = 0
    FN = 0
    FP = 0
    for i,j in zip(ts_lb,answer):
        if j==1 and i==1:
            TP += 1
        elif(j==1 and i==0):
            FN += 1
        elif(j==0 and i==1):
            FP += 1
        elif(j==0 and i==0):
            TN += 1
    Accuracy = (TP + TN)/(TP + FP + TN + FN)
    Precision = TP/(TP + FP)
    Recall = TP/(TP + FN)
    f1_score = (2*Precision*Recall)/(Precision + Recall)
    return Accuracy, Precision, Recall, f1_score

In [3]:
def check_for_leaf(df,counter, min_samples, max_depth):
    unique_classes = np.unique(df)
    if len(unique_classes) == 1 or len(df)<=min_samples or counter==max_depth:
        labelcol = df
        uniq_cls, cnt = np.unique(labelcol, return_counts=True)
        classification = unique_classes[cnt.argmax()]
        return classification
    else:
        return False 

In [4]:
def gini_imp_test(df, col_index):
    df.reset_index(inplace = True, drop = True)
    classes = df.iloc[:,-1]
    feature = df.iloc[:,col_index]
    if len(feature.unique()) == 2:
        gini_imp = 0
        for i in np.unique(feature):
            idx = np.where(feature == i)
            label = classes.loc[idx].values
            a, b = np.unique(label, return_counts = True)
            list1 = [(i/sum(b))**2 for i in b]
            prob = 1 - sum(list1)
            wt = len(idx[0]) / df.shape[0]
            gini_imp += wt * prob
        return gini_imp, i
    else:
        label = np.sort(feature.unique())[1:-1]
        best_gini_imp = float('inf')
        split_val = 0
        for i in label:
            idx1 = np.where(feature > i)
            idx2 = np.where(feature <= i)
            if len(idx1[0]) > 2 and len(idx2[0]) > 2:
        
                b1, b1cnt = np.unique(classes.loc[idx1].values, return_counts = True)
                b2, b2cnt = np.unique(classes.loc[idx2].values, return_counts = True)
                list1 = [(i/sum(b1cnt))**2 for i in b1cnt]
                list2 = [(i/sum(b2cnt))**2 for i in b2cnt]
                prob1 = 1 - sum(list1)
                prob2 = 1 - sum(list2)
                gini = ((sum(b1cnt)/df.shape[0])*prob1) + ((sum(b2cnt)/df.shape[0])*prob2) 
                if gini < best_gini_imp:
                    best_gini_imp = gini
                    split_val = i
                else:
                    continue 
        return best_gini_imp, split_val

In [17]:
def best_node(df):
    best_gini_imp = float('inf')
    value = 0
    col = 0
    for i in range(df.iloc[:,:-1].shape[1]):
        gini, val = gini_imp_test(df, i) 
        if gini < best_gini_imp:
            best_gini_imp = gini
            value = val
            col = i
    return col, value, best_gini_imp

In [6]:
def split_df(df, col_index, split_val):
    feature = df.iloc[:,col_index]
    if feature.dtypes == object:
        temp1 = df[df.iloc[:,col_index] == split_val]
        temp2 = df[df.iloc[:,col_index] != split_val]
        return temp1, temp2
    elif feature.dtypes != object:
        temp1 = df[df.iloc[:,col_index] <= split_val]
        temp2 = df[df.iloc[:,col_index] > split_val]
        temp1.reset_index(inplace = True, drop = True)
        temp2.reset_index(inplace = True, drop = True)
        return temp1, temp2

In [7]:
def check_purity(data):
    
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False

In [8]:
def classify_data(data):
    
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification

In [14]:
def decision_tree(df, counter = 0, min_samples = 1, max_depth = 5):
    if (check_purity(df.values)) or (len(df) < min_samples) or (counter == max_depth):
        classification = classify_data(df.values)
        
        return classification
      
    else:
        counter += 1
        column, value, gini = best_node(df)
        print("Depth "+str(counter)+" Impurity measure: "+str(gini))
        branch1, branch2 = split_df(df, column, value)
        if len(branch1) == 0 or len(branch2) == 0:
            classification = classify_data(df.values)
            return classification
        query = "{} <= {}".format(column, value)
        branch = {query: []}

        left_branch = decision_tree(branch1, counter)
        right_branch = decision_tree(branch2, counter)

        if left_branch == right_branch:
            branch = left_branch
        else:
            branch[query].append(left_branch)
            branch[query].append(right_branch)
        return branch

In [10]:
def predict(model, test_data):
    cls = []
    for i in range(len(test_data)):
        t = model
        col,_,val = list(t.keys())[0].split()
        col = int(col)
        try:
            val = float(val)
        except:
            val = str(val)
        key = list(t.keys())[0]
        key_val = t[key]
        while True: 
            if test_data.iloc[i,col] <= val:
                t = t[key][0]
                if type(t) != dict:
                    cls.append(t)
                    break
                else:
                    col,_,val = list(t.keys())[0].split()
                    col = int(col)
                    try:
                        val = float(val)
                    except:
                        val = str(val)
                    key = list(t.keys())[0]
                    key_val = t[key]
            else:
                t = t[key][1]
                if type(t) != dict:
                    cls.append(t)
                    break
                else:
                    col,_,val = list(t.keys())[0].split()
                    col = int(col)
                    try:
                        val = float(val)
                    except:
                        val = str(val)
                    key = list(t.keys())[0]
                    key_val = t[key]
    cls = [int(i) for i in cls]
    test_data["Class"] = cls
    return test_data

In [11]:
def k_fold(df):
    k = int(input("Enter k value: "))
    metrics_list = []
    for i in range(k):
        splitdfs = np.array_split(df, k)
        test = splitdfs[i]
        del(splitdfs[i])
        train = pd.concat(splitdfs)
        test.reset_index(inplace = True, drop = True)
        train.reset_index(inplace = True, drop = True) 
        actual = test.iloc[:,-1]
        test = test.iloc[:,:-1]
        model = decision_tree(train)
        results = predict(model, test)
        Accuracy, Precision, Recall, f1_score = metrics(actual, results["Class"])
        metrics_list.append([Accuracy, Precision, Recall, f1_score])
    metrics_list = np.array(metrics_list)
    metrics_list = np.mean(metrics_list, axis = 0)
    print("Accuracy: ",metrics_list[0])
    print("Precision: ",metrics_list[1])
    print("Recall: ",metrics_list[2])
    print("f1_score: ",metrics_list[3])
    return metrics_list

In [12]:
df1 = pd.read_csv("project3_dataset1.txt", sep = '\t', header=None)
results1 = k_fold(df1)
results1

Enter k value: 5
Accuracy:  0.9419034311442323
Precision:  0.9080338872450474
Recall:  0.9418506280408254
f1_score:  0.9232578084172698


array([0.94190343, 0.90803389, 0.94185063, 0.92325781])

In [13]:
df2 = pd.read_csv("project3_dataset2.txt", sep = '\t', header=None)
results2 = k_fold(df2)
results2

Enter k value: 5
Accuracy:  0.5799672744273024
Precision:  0.4794155844155844
Recall:  0.4151728245344544
f1_score:  0.4388484949853003


array([0.57996727, 0.47941558, 0.41517282, 0.43884849])

# Demo

In [48]:
demo = pd.read_csv("project3_dataset4.txt", sep = '\t', header=None)

In [49]:
decision_tree(demo)

Depth 1 Impurity measure: 0.3673469387755103
Depth 2 Impurity measure: 0.19047619047619047
Depth 3 Impurity measure: 0.3333333333333333
Depth 4 Impurity measure: 0.0
Depth 2 Impurity measure: 0.21428571428571427
Depth 3 Impurity measure: 0.0
Depth 3 Impurity measure: 0.0


{'2 <= normal': [{'3 <= weak': [1, {'1 <= mild': [1, {'0 <= rain': [0, 1]}]}]},
  {'0 <= rain': [{'3 <= weak': [1, 0]}, {'0 <= sunny': [0, 1]}]}]}